In [ ]:
!pip install duckdb

In [28]:
import duckdb

In [29]:
con = duckdb.connect(database =':memory:', read_only=False )

In [30]:
query = "select * from 'customers.csv'"

In [31]:
result = con.execute(query).fetchdf()

In [32]:
result.head(2)

,customer_id,first_name,last_name,email,created_at,created_date,status,country
0,1001,John,Doe,john.doe@example.com,2024-01-15 08:23:45,2024-01-15,active,US
1,1002,Emily,Smith,emily.smith@example.co.uk,2024-02-10 14:12:30,2024-02-10,active,UK


In [33]:
result.shape

(10, 8)

In [82]:
filename = 'Titanic-Dataset.csv'

In [83]:
query = "select * from 'Titanic-Dataset.csv'"

In [84]:
titanic_result = con.execute(query).fetchdf()

In [85]:
titanic_result.head(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,None,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


In [86]:
titanic_result.shape

(891, 12)

In [87]:
titanic_result.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [88]:
query = "select * from read_json_auto('products.json')"

In [89]:
product_result = con.execute(query).fetchdf()

In [90]:
product_result.head()

,id,name,price,category,in_stock
0,1,Laptop,35000,Electronics,True
1,2,Phone,18000,Electronics,False
2,3,Desk Chair,4500,Furniture,True


In [91]:
result.columns

Index(['customer_id', 'first_name', 'last_name', 'email', 'created_at',
       'created_date', 'status', 'country'],
      dtype='object')

In [93]:
query = """
select
  customer_id,
  first_name,
  last_name,
  email,
  created_date,
from 'customers.csv'
where status = 'active'
 and created_date >= '2024-01-01'
 and country = 'TH'
"""

In [116]:
customer_resultf = con.execute(query).fetchdf()

In [117]:
customer_resultf.head()

,customer_id,customer_name,segment,order_count,total_amount,avg_amount
0,C003,Acme Corp,SMB,1,20.0,20.0
1,C002,Jane Smith,Corporate,0,NaN,NaN
2,C010,Small Biz,SMB,0,NaN,NaN
3,C001,John Doe,Consumer,1,50.0,50.0
4,C007,Cancelled Cust,Consumer,1,100.0,100.0


In [118]:
titanic_result.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [119]:
query = """
select
  PassengerId,
  Name,
  Age,
  Fare,
  case
    when Survived = 1 then 'survived'
    else 'Not survived'
  end as survived_status
from 'Titanic-Dataset.csv'
"""

In [120]:
transformed_data = con.execute(query).fetchdf()

In [121]:
transformed_data.head()

,PassengerId,Name,Age,Fare,survived_status
0,1,"Braund, Mr. Owen Harris",22.0,7.2500,Not survived
1,2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,71.2833,survived
2,3,"Heikkinen, Miss. Laina",26.0,7.9250,survived
3,4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35.0,53.1000,survived
4,5,"Allen, Mr. William Henry",35.0,8.0500,Not survived


In [122]:
query = "select * from 'orders.csv'"

order_result = con.execute(query).fetchdf()

order_result.columns

Index(['order_id', 'customer_id', 'amount', 'order_date', 'status'], dtype='object')

In [123]:
query = "select * from 'new_customers.csv'"

customer_result2 = con.execute(query).fetchdf()

customer_result2.columns

Index(['customer_id', 'customer_name', 'segment', 'is_active'], dtype='object')

In [124]:
query = """
select
  c.customer_id,
  c.customer_name,
  c.segment,
  count(distinct o.order_id) as order_count,
  sum(o.amount) as total_amount,
  avg(o.amount) as avg_amount
from 'new_customers.csv' c
 left join 'orders.csv' o
  on c.customer_id = o.customer_id
   and o.status = 'cancelled'
where c.is_active = true
group by 1,2,3
"""

In [125]:
customer_metric = con.execute(query).fetchdf()

In [126]:
customer_metric.head()

,customer_id,customer_name,segment,order_count,total_amount,avg_amount
0,C001,John Doe,Consumer,1,50.0,50.0
1,C007,Cancelled Cust,Consumer,1,100.0,100.0
2,C005,Global LLC,Enterprise,0,NaN,NaN
3,C009,Future Buyer,Enterprise,0,NaN,NaN
4,C006,NoOrders Co,SMB,0,NaN,NaN


In [141]:
query = """
with
active_orders as (
  select
    customer_id,
    count(*) as order_count,
    sum(amount) as total_amount,
    avg(amount) as avg_amount,
  from 'orders.csv'
  where order_date >= date_add(current_date(), interval '-12 month')
  group by 1
 ),

 active_customers as (
  select
    customer_id,
    customer_name,
    segment,
  from 'new_customers.csv'
  where is_active = true
),

final_result as (
  select
    ac,*,
    COALESCE(ao.order_count, 0) as order_count,
    COALESCE(ao.total_amount, 0) as total_amount,
    ao.avg_amount as avg_amount
from active_customers ac
  left join active_orders ao
 on ac.customer_id = ao.customer_id
)

select * from final_result
"""

In [142]:
result2 = con.execute(query).fetchdf()

In [143]:
result2.head()

,ac,customer_id,customer_name,segment,customer_id_1,order_count,total_amount,avg_amount,order_count_1,total_amount_1,avg_amount_1
0,"{'customer_id': 'C007', 'customer_name': 'Canc...",C007,Cancelled Cust,Consumer,C007,1,100.0,100.0,1,100.0,100.0
1,"{'customer_id': 'C009', 'customer_name': 'Futu...",C009,Future Buyer,Enterprise,C009,1,1000.0,1000.0,1,1000.0,1000.0
2,"{'customer_id': 'C010', 'customer_name': 'Smal...",C010,Small Biz,SMB,C010,1,45.0,45.0,1,45.0,45.0
3,"{'customer_id': 'C001', 'customer_name': 'John...",C001,John Doe,Consumer,None,<NA>,NaN,NaN,0,0.0,NaN
4,"{'customer_id': 'C002', 'customer_name': 'Jane...",C002,Jane Smith,Corporate,None,<NA>,NaN,NaN,0,0.0,NaN


In [145]:
query = """
WITH enriched_orders AS (
    SELECT
        o.*,
        c.customer_name,
        c.segment
    FROM orders.csv o
    LEFT JOIN new_customers.csv c
        ON o.customer_id = c.customer_id
)
SELECT
    order_id,
    customer_id,
    customer_name,
    order_date,
    amount,

    -- 1. ลำดับของ order ของลูกค้าแต่ละคน
    ROW_NUMBER() OVER (
        PARTITION BY customer_id
        ORDER BY order_date
    ) as order_sequence,

    -- 2. อันดับยอดขายในเดือนนั้น
    RANK() OVER (
        PARTITION BY strftime('%Y-%m', order_date)
        ORDER BY amount DESC
    ) as monthly_sales_rank,

    -- 3. ยอดสะสมของลูกค้า
    SUM(amount) OVER (
        PARTITION BY customer_id
        ORDER BY order_date
    ) as running_total,

    -- 4. ค่าเฉลี่ยเคลื่อนที่ 4 orders
    AVG(amount) OVER (
        PARTITION BY customer_id
        ORDER BY order_date
        ROWS BETWEEN 3 PRECEDING AND CURRENT ROW
    ) as moving_avg_4_orders,

    -- 5. ยอด order ก่อนหน้า
    LAG(amount, 1, 0) OVER (
        PARTITION BY customer_id
        ORDER BY order_date
    ) as previous_order,

    -- 6. ยอด order ถัดไป
    LEAD(amount, 1, 0) OVER (
        PARTITION BY customer_id
        ORDER BY order_date
    ) as next_order,

    -- 7. ยอด order แรกของลูกค้า
    FIRST_VALUE(amount) OVER (
        PARTITION BY customer_id
        ORDER BY order_date
    ) as first_order_amount

FROM enriched_orders
ORDER BY customer_id, order_date;
"""

In [146]:
window_analysis = con.execute(query).fetchdf()

In [147]:
window_analysis.head()

,order_id,customer_id,customer_name,order_date,amount,order_sequence,monthly_sales_rank,running_total,moving_avg_4_orders,previous_order,next_order,first_order_amount
0,O1001,C001,John Doe,2024-01-15,120.5,1,1,120.5,120.500000,0.0,75.0,120.5
1,O1002,C001,John Doe,2024-02-20,75.0,2,1,195.5,97.750000,120.5,50.0,120.5
2,O1003,C001,John Doe,2024-03-05,50.0,3,1,245.5,81.833333,75.0,0.0,120.5
3,O1004,C002,Jane Smith,2024-04-25,200.0,1,1,200.0,200.000000,0.0,150.0,200.0
4,O1005,C002,Jane Smith,2024-05-01,150.0,2,1,350.0,175.000000,200.0,0.0,200.0
